*Author:wxz*  
*2018/08/06*  
*Email:876688461@qq.com*

 # Work
目前正在做一个把`Pytorch`框架的推荐算法改成`Mxnet`框架。  
`mxnet`框架发展得很快速，目前支持的编程语言有Python,Scala,R, Julia, Perl, Cpp。支持GPU编程，也支持多GPU编程。 支持Linux和MacOS，但是WindowOS还不支持。  
改编的过程还算顺利，因为两种框架的层的名字、定义基本相同，还有数据集类的定义也相同，基本不用改动，但是同时也碰到了很多坑，这些坑是这两个框架之间很不一样的地方，其中有一个初始化的坑卡了差不多2天。废话不多说，摆坑。

# 1.初始化方式不一样
pytorch的初始化可以在网络定义的时候直接初始化，但是mxnet的初始化必须在建立对象之后，由对象执行initialize()函数才能够初始化。下面给出代码逐步分析，为了让代码简洁，特意删去网络的前传过程，仅仅给出初始化过程：`__init__()`
## Pytorch

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import pdb

class NeuMF(nn.Module):
    def __init__(self, nb_users, nb_items,
                 mf_dim):
        # TODO: regularization?
        super(NeuMF, self ).__init__()
        self.mf_user_embed = nn.Embedding(nb_users, mf_dim)
        self.mf_item_embed = nn.Embedding(nb_items, mf_dim)
        self.mlp_user_embed = nn.Embedding(nb_users, 1)
        self.mlp_item_embed = nn.Embedding(nb_items, 1)

        # pytorch在函数定义时就可以初始化函数，也就是说这个NeuMF类生成一个
        # 对象时就可以直接拥有初始化的权重。
        self.mf_user_embed.weight.data.normal_(0., 0.01)
        self.mf_item_embed.weight.data.normal_(0., 0.01)
        self.mlp_user_embed.weight.data.normal_(0., 0.01)
        self.mlp_item_embed.weight.data.normal_(0., 0.01)

nb_users, nb_items = 1,1
mf_dim = 1

net = NeuMF(nb_users, nb_items, mf_dim) # net现在已经初始化了，可以直接输入数据


## Mxnet

In [ ]:
import numpy as np
from mxnet.gluon import nn
from mxnet import nd
import mxnet as mx

class NeuMF(nn.HybridBlock): #if using nn.Hybridblock, it will generate static graph
    def __init__(self, nb_users, nb_items,
                 mf_dim): 
        super(NeuMF, self).__init__()
        with self.name_scope():
            # 这里指定了四个层的初始化方式：mx.init.Normal()
            # 但是只是指定了方式而已，还未初始化
            self.mf_user_embed = nn.Embedding(nb_users, mf_dim,
                                          weight_initializer=mx.init.Normal())
            self.mf_item_embed = nn.Embedding(nb_items, mf_dim,
                                          weight_initializer=mx.init.Normal())
            self.mlp_user_embed = nn.Embedding(nb_users, 1,
                                           weight_initializer=mx.init.Normal())
            self.mlp_item_embed = nn.Embedding(nb_items, 1,
                                           weight_initializer=mx.init.Normal())
nb_users, nb_items = 1,1
mf_dim = 1

net = NeuMF(nb_users, nb_items,
            mf_dim) # net现在还未初始化
net.initialize() # 使用了initialize()函数之后，net才算是真正初始化

# 2.自动求导的方式不一样
## Pytorch

In [ ]:
import torch
# pytorch里是将变量变成torch.autograd.Variable类型来自动求导的

user = torch.autograd.Variable(user, requires_grad=False)
item = torch.autograd.Variable(item, requires_grad=False)
label = torch.autograd.Variable(label, requires_grad=False)

# pretend that the model has been well trained
outputs = model(user, item)
loss = criterion(outputs, label)
loss.backward()

## Mxnet

In [ ]:
import mxnet

ctx = mxnet.cpu()
user = nd.array(user,ctx=ctx)
item = nd.array(item,ctx=ctx)
label = nd.array(label,ctx=ctx)

# compute the gradient automatically
# mxnet里是将变量放到with autograd.record():
# 之下使其获得自动求导的能力
with autograd.record():
    outputs = model(user, item)
    loss = mxnet_criterion(outputs, label.T)

loss.backward()
trainer.step(bs)

# 3.使用GPU编程的定义操作不一样

## Pytorch


In [ ]:
# 数据
import torch 

所有数据都默认放在cpu里

In [ ]:
a = torch.tensor([1,2,3])

# 选择cpu还是gpu,这里选择gpu
use_cuda = True
device = torch.device("cuda" if use_cuda else "cpu")

# 把数据放到gpu里
a = a.to(device)

在编写深度学习网络时，要把网络模型、损失函数、优化准则也一并放到
gpu里才行  
假设model, loss, criterion分别是定义好的网络模型、损失函数、
优化准则，由于没有预先定义，下面的代码运行会出错

In [ ]:
# 存放到gpu里
model = model.to(device)
loss = loss.to(device)
criterion = criterion.to(device)

## Mxnet
在mxnet的`gluon`模块里，几乎所有的类在定义是都会具有ctx这个参数，  
我们在定义我们的变量时可以选择把其放到cpu还是gpu

In [ ]:
# 数据
import mxnet
import numpy as np

选择cpu还是gpu,这里选择gpu  
注意mxnet只能支持Nvidia的显卡  
不支持AMD的显卡  

In [ ]:
use_cuda = True
ctx= mx.gpu() if use_cuda else mx.cpu()

# 默认放在cpu里
a = mxnet.nd.array([1,2,3])

# 使用gpu
a = mxnet.nd.array([1,2,3], ctx=ctx)

**mxnet里不用把网络、损失函数、优化准则放到gpu里**

------------------
若在定义数据时忘记把数据放到gpu里而又想在gpu里  
运算怎么办呢

使用copyto和as_in_context都可以把数据转化到gpu

In [ ]:
b = mxnet.nd.array([1,2,3])

b_gpu = b.copyto(mxnet.gpu())
b_gpu1 = b.as_in_context(mxnet.gpu())